In [1]:
# imports
# from transformers import pipeline
from helpers import Video_loader, Image_loader, Captioner, Audio_transcriber

In [2]:
VIDEOS_DIRECTORY = "videos"
FRAME_FREQUENCY = 6000

In [3]:
# video captions section
video_loader = Video_loader(VIDEOS_DIRECTORY, FRAME_FREQUENCY)

captions_dict = video_loader.process()
# need to tune this up but for now it's whatever - we have frame visual captions now

print("------------------------------------------------------------------------------------------\n END OF HELPERS \n")
print(captions_dict)

Init video loader
Make frames
Storing in videos\Why the new MacBook Air is a BIG deal.mp4_images
Start image loading
Init image loader
Init captioner


All model checkpoint layers were used when initializing TFBlipForConditionalGeneration.

All the layers of TFBlipForConditionalGeneration were initialized from the model checkpoint at Salesforce/blip-image-captioning-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBlipForConditionalGeneration for predictions without further training.


Start image get captions


c:\Users\david\anaconda3\envs\research-env\lib\site-packages\transformers\generation\tf_utils.py:854: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


------------------------------------------------------------------------------------------
 END OF HELPERS 

{'videos\\Why the new MacBook Air is a BIG deal.mp4_images': [[{'generated_text': 'a man with glasses and a beard leaning on a yellow bench'}], [{'generated_text': 'a man sitting at a table with a laptop'}], [{'generated_text': 'a bunch of different screens and screens'}]]}


In [3]:
# audio captions section
audio_transcriber = Audio_transcriber(VIDEOS_DIRECTORY)
print(audio_transcriber.transcribe_video_folder(VIDEOS_DIRECTORY))

Transcribing video folder: videos
Filenames: 
[<DirEntry 'FP3 Highlights _ 2023 Hungarian Grand Prix.mp4'>, <DirEntry 'Why the new MacBook Air is a BIG deal.mp4'>]
Make folders and save audio
Storing in videos\FP3 Highlights _ 2023 Hungarian Grand Prix.mp4_audio
Transcribing video <DirEntry 'FP3 Highlights _ 2023 Hungarian Grand Prix.mp4'>
Audio split!
videos\FP3 Highlights _ 2023 Hungarian Grand Prix.mp4_audio\chunk1.wav : Final practice in budapest have a seven-time world champion lewis hamilton top of the timing table for the fine last time that one minute 17811 after a disappointing friday practice the britain had reflected on his performance suggesting the w14 was an eight-time winner here house and seems to have used all that experience with a truck to extract the maximum the car this time out silver arrows teammate george russell had been a little left has the mystic coming into the final practice session in which he finish his dick some 3/10 back from the leader current drivers

KeyboardInterrupt: 

In [ ]:
# fusion section
# steps to achieve:
# - combine string per video, do word embeddings with GloVe
# - build fusion layer

In [ ]:
# model compilation
# steps to achieve:
# - implement transformer model to analyse all text
# - decide on loss
# - decide on optimiser
# - compile model

In [ ]:
# training
# steps to achieve:
# - download all data
# - run preprocessing on all of the videos

In [ ]:
# testing